In [1]:
from env2048.vectenv import Vectorized2048Env
from env2048.vectmcts import Vectorized2048MCTSLazy
from az_resnet import AZResnet, AZResnetArchitectureParameters
import torch

/Users/marshingjay/miniconda3/envs/python3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

from env2048.vecttrainer import VectorizedTrainer


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
num_boards = 10
v = Vectorized2048Env(num_boards, device)
model = AZResnet(AZResnetArchitectureParameters(
    input_size=torch.Size((1, 4, 4)),
    policy_size=4,
    res_channels=8,
    res_blocks=2, 
    value_head_res_channels=8,
    value_head_res_blocks=2,
    policy_head_res_channels=8,
    policy_head_res_blocks=2,
    kernel_size=3,
    policy_fc_size=32,
    value_fc_size=32
))
mc = Vectorized2048MCTSLazy(v, model, 1)
v.reset()
trainer = VectorizedTrainer(mc, model, 5, 5)

In [3]:
def run_for_steps(steps):
    for _ in range(steps):
        boards, actions, terminated = trainer.run_step()

In [4]:
%prun -s tottime run_for_steps(100)

         2040434 function calls (1810334 primitive calls) in 8.349 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    58600    3.618    0.000    3.618    0.000 {built-in method torch.conv2d}
    39000    1.175    0.000    1.175    0.000 {built-in method torch.batch_norm}
     6200    0.469    0.000    2.208    0.000 vectenv.py:52(get_legal_moves)
     6200    0.267    0.000    0.338    0.000 vectenv.py:95(rotate_by_amnts)
     3100    0.259    0.000    0.426    0.000 vectenv.py:104(merge)
187200/2600    0.219    0.000    4.709    0.002 module.py:1124(_call_impl)
41600/10400    0.148    0.000    4.663    0.000 container.py:137(forward)
   340600    0.146    0.000    0.146    0.000 module.py:1194(__getattr__)
    39000    0.142    0.000    1.496    0.000 batchnorm.py:134(forward)
    40500    0.126    0.000    0.126    0.000 {built-in method torch.any}
    15600    0.116    0.000    4.018    0.000 az_resnet.py:36(forward)
  